#Set-Up

In [1]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo and Coin-OR solvers.
#for reference, see https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.02-Running-Pyomo-on-Google-Colab.html#installing-pyomo-and-solvers

%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *

In [2]:
import pandas as pd

#Code that solves one small assignment problem


In [29]:
#write code that solves one instance of the assignment problem
times = [[193,171,51],
         [189,128,35],
         [39,174,46]]
#first index is the person, second index is the task, times[i][j] denotes the time it takes person i to complete task j

#declare a concrete model
model = ConcreteModel()

num_people = 3 #use i to index the people, this is the first index
num_tasks = 3 #use j to index the tasks, this is the second index

#declare the decision variables
model.x = Var(range(num_people), range(num_tasks), domain=NonNegativeReals) #NOTE: This is how you define 2-dimensional decision variables, model.x[i,j]

#set the objective
model.Objective = Objective(expr = sum(model.x[i,j]*times[i][j] for i in range(num_people) for j in range(num_tasks)), sense = minimize) #this is how you sum up over multiple indices

#set the people constraints
model.PeopleConstraints = ConstraintList() #declaring an empty list of constraints
for i in range(num_people): #add one constraint per person
    model.PeopleConstraints.add(expr = sum(model.x[i,j] for j in range(num_tasks)) <= 1)

#set the task constraints
model.TaskConstraints = ConstraintList()
for j in range(num_tasks): #adding one constraint per task
    model.TaskConstraints.add(expr = sum(model.x[i,j] for i in range(num_people)) >= 1)

#model.pprint()


In [30]:
#solve the model
opt = SolverFactory('cbc')
opt.solve(model, tee = False)

{'Problem': [{'Name': 'unknown', 'Lower bound': 218.0, 'Upper bound': 218.0, 'Number of objectives': 1, 'Number of constraints': 6, 'Number of variables': 9, 'Number of nonzeros': 9, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.0, 'Wallclock time': 0.0, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': None, 'Number of created subproblems': None}, 'Black box': {'Number of iterations': 5}}, 'Error rc': 0, 'Time': 0.022170066833496094}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [31]:
#print solution
print("objective:", model.Objective())
for i in range(num_people):
    for j in range(num_tasks):
        if model.x[i,j]() > 0:
            print("person", i, "gets task", j)

objective: 218.0
person 0 gets task 2
person 1 gets task 1
person 2 gets task 0


In [32]:
#alternative solution output
print("objective:", model.Objective())
assignment_matrix = [[model.x[i,j]() for j in range(num_tasks)] for i in range(num_people)]
assignment_matrix

objective: 218.0


[[0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [1.0, 0.0, 0.0]]

In [33]:
#converting assignments to dataframe
pd.DataFrame(assignment_matrix, index = ["person1", "person2", "person3"], columns = ["task1","task2","task3"])

,task1,task2,task3
person1,0.0,0.0,1.0
person2,0.0,1.0,0.0
person3,1.0,0.0,0.0


#Option A: Solving one large (47x47) assignment problem
Here is [the excel data](https://uwmadison.box.com/shared/static/m21ihwgu4aqvfoz5s1ozx980p7zuk5qb.xlsx).

In [9]:
#load in data (located on the first sheet)
df2 = pd.read_excel('LargeAssignmentProblemData.xlsx', sheet_name='Largeproblemdata')
df2.head()

,PersonNum,Task1,Task2,Task3,Task4,Task5,Task6,Task7,Task8,Task9,...,Task38,Task39,Task40,Task41,Task42,Task43,Task44,Task45,Task46,Task47
0,1,328,240,99,244,310,83,209,76,66,...,330,134,182,84,341,193,327,54,160,312
1,2,153,111,96,308,297,114,51,178,167,...,114,59,336,91,308,104,253,199,62,118
2,3,82,246,163,277,68,55,82,232,313,...,86,113,95,151,160,185,96,320,232,254
3,4,149,171,81,172,163,298,319,100,155,...,101,337,222,161,192,207,190,292,53,133
4,5,201,211,224,323,133,112,165,186,116,...,144,195,307,253,280,237,232,277,204,130


In [39]:
times = df2.values.tolist()

# Declare a concrete model
model = ConcreteModel()

# Number of people and tasks
num_people = 47
num_tasks = 47

# Declare the decision variables
model.x = Var(range(num_people), range(num_tasks), domain=NonNegativeReals)

# Set the objective (minimizing the total time)
model.Objective = Objective(expr = sum(model.x[i, j] * times[i][j] for i in range(num_people) for j in range(num_tasks)), sense = minimize)

# Set the people constraints
model.PeopleConstraints = ConstraintList()
for i in range(num_people):
    model.PeopleConstraints.add(expr = sum(model.x[i, j] for j in range(num_tasks)) <= 1)

# Set the task constraints
model.TaskConstraints = ConstraintList()
for j in range(num_tasks):
    model.TaskConstraints.add(expr = sum(model.x[i, j] for i in range(num_people)) >= 1)

In [40]:
#solve the model
opt = SolverFactory('cbc')
opt.solve(model, tee = False)

{'Problem': [{'Name': 'unknown', 'Lower bound': 2735.0, 'Upper bound': 2735.0, 'Number of objectives': 1, 'Number of constraints': 94, 'Number of variables': 2209, 'Number of nonzeros': 2209, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.02, 'Wallclock time': 0.02, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': None, 'Number of created subproblems': None}, 'Black box': {'Number of iterations': 106}}, 'Error rc': 0, 'Time': 0.03608536720275879}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [41]:
#print solution
print("objective:", model.Objective())
for i in range(num_people):
    for j in range(num_tasks):
        if model.x[i,j]() > 0:
            print("person", i, "gets task", j)

objective: 2735.0
person 0 gets task 0
person 1 gets task 7
person 2 gets task 6
person 3 gets task 46
person 4 gets task 36
person 5 gets task 11
person 6 gets task 34
person 7 gets task 35
person 8 gets task 13
person 9 gets task 43
person 10 gets task 40
person 11 gets task 24
person 12 gets task 12
person 13 gets task 3
person 14 gets task 37
person 15 gets task 19
person 16 gets task 32
person 17 gets task 26
person 18 gets task 42
person 19 gets task 8
person 20 gets task 21
person 21 gets task 5
person 22 gets task 16
person 23 gets task 44
person 24 gets task 20
person 25 gets task 15
person 26 gets task 10
person 27 gets task 9
person 28 gets task 29
person 29 gets task 22
person 30 gets task 4
person 31 gets task 30
person 32 gets task 28
person 33 gets task 31
person 34 gets task 45
person 35 gets task 38
person 36 gets task 41
person 37 gets task 14
person 38 gets task 23
person 39 gets task 1
person 40 gets task 25
person 41 gets task 18
person 42 gets task 17
person 43 ge

In [45]:
#alternative solution output
print("objective:", model.Objective())
assignment_matrix = [[model.x[i,j]() for j in range(num_tasks)] for i in range(num_people)]
#assignment_matrix

objective: 2735.0


#Option B: Solving 50 small (3x3) assignment problems
Here is [the excel data](https://uwmadison.box.com/shared/static/m21ihwgu4aqvfoz5s1ozx980p7zuk5qb.xlsx).

In [ ]:
#load in data (located on the first sheet)
df = pd.read_excel('AssignmentProblemData.xlsx', sheet_name='50problemsdata')
df.head()